In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Predict short term movements in stock prices using news and sentiment data provided by RavenPack



** We have to develop a model that predicts stock price movements using sentiment data provided by RavenPack. Traders, analysts and investors are always looking for techniques to better predict price movements.  Knowing whether a security will increase or decrease allows traders to make better investment decisions and manage risk more effectively.**

** Objective **

<p>We have to predict the percentage change in a financial instrument at a time 2 hours in the future.  The data represents features of various financial securities (198 in total) recorded at 5-minute intervals throughout a trading day. Features' names or the specific dates are not provided.</p>

* Each variable named O1, O2, O3, etc. (the outputs) represents a percent change in the value of a security.  

* Each variable named I1, I2, I3, etc. (the inputs) represents a feature. The underlying securities and features represented by these anonymized names are the same across all files (e.g. O1 will always be the same stock).

* We are provided the outputs as a relative percentage compared to the previous day's closing price.  The first line of each data file represents the previous close. For example, if a security closed at 1dollar the previous day and opened at 2 dollar the next day, the first output would be 0, then 100.  All output values are computed relative to the previous day's close. The timestamps within each file are as follows (ignoring the header row):

* Line 1 = Outputs and inputs at previous day's close (4PM ET)
* Line 2 = Outputs and inputs at current day's open (9:30AM ET)
* Line 3 = Outputs and inputs at 9:35AM ET
* ...
* Line 55 = Outputs and inputs at 1:55PM ET

> We have to predict the outputs 2 hours later, at 4PM ET.